In [ ]:
%load_ext autoreload
%autoreload 2
# default_exp indexers.indexer

In [ ]:
# export
from integrators.data.schema import *
from integrators.pod.client import PodClient

# Indexer

In [ ]:
# export
class IndexerBase(Indexer):

    def __init__(self, indexerClass=None, *args, **kwargs):
        if indexerClass is None: indexerClass=self.__class__.__name__
        super().__init__(indexerClass=indexerClass, *args, **kwargs)    
    
    def populate(self, api, updated_items, new_nodes, edges=False):
        for item in new_nodes:
            item.update(api, edges=False)
        if edges:
            for item in new_nodes:
                item.update_edges(api)

        for item in updated_items:
            item.update(api)
    
            
def get_indexer_run_data(client, indexer_run):
    if indexer_run.targetDataType is None:
        print("No targetDataType defined")
        return None

    # get all items with the specified type
    else:
        return client.search_by_fields({"_type": indexer_run.targetDataType})

# Running your own indexer

When we run an indexer we have four steps. 1) Get the indexer and indexer run based on the run uid. 2) run the indexer 3) populate the graph with the new information. To mock that, first we create a client and add some toy data.

In [ ]:
from integrators.indexers.geo.geo_indexer import GeoIndexer

client = PodClient(database_key="0" * 64, owner_key="1" * 64)

location = Location.from_data(latitude=-37.81, longitude=144.96)
address = Address.from_data()
indexer = Indexer.from_data(indexerClass="GeoIndexer", name="GeoIndexer")
indexer_run = IndexerRun.from_data(progress=0, targetDataType="Address")

for x in [location, address, indexer, indexer_run]: client.create(x)

In [ ]:
edge_succes = client.create_edge(Edge(indexer_run, indexer, "indexer"))
edge_succes = client.create_edge(Edge(location, address, "location"))

assert edge_succes

Now we start with the setting we would normally have: some memri client makes a call to the pod to execute an indexer run. Lets start by getting the indexer and the indexer run.

In [ ]:
indexer_run = client.get(indexer_run.uid)
indexer = indexer_run.indexer[0]

Next, we retrieve the data, which was specified in the client by the `targetDataType`.

In [ ]:
data = get_indexer_run_data(client, indexer_run)
data[:3]

[Address (#2)]

In [ ]:
updated_items, new_items = indexer.index(client, indexer_run)
updated_items, new_items

1 items found to index
Loading formatted geocoded file...
updating IndexerRun (#4)


([Address (#2)], [Country (#None)])

In [ ]:
indexer.populate(client, updated_items, new_items)

creating Country (#None)
updating Address (#2)


# Export -

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted basic.ipynb.
Converted index.ipynb.
Converted indexers.GeoIndexer.ipynb.
Converted indexers.indexer.ipynb.
Converted itembase.ipynb.
Converted pod.client.ipynb.
